<a href="https://colab.research.google.com/github/Prady96/Pothole-Detection/blob/avi_testing/Project_update_24th_05_07pm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#lists the content of your google drive
!ls "/content/drive/My Drive/Potholes_tataInnoverse/mainFile/train_val_22ndFeb/"

main_DataSETS_TRAIN.zip  main_DATASET_VAL.zip


In [3]:
import shutil
shutil.copy('/content/drive/My Drive/Potholes_tataInnoverse/mainFile/train_val_22ndFeb/main_DATASET_VAL.zip', '/content/val.zip')

'/content/val.zip'

In [4]:
import shutil
shutil.copy('/content/drive/My Drive/Potholes_tataInnoverse/mainFile/train_val_22ndFeb/main_DataSETS_TRAIN.zip', '/content/train.zip')

'/content/train.zip'

In [0]:
!mkdir customImages

In [0]:
!mv train.zip customImages/

In [0]:
!mv val.zip customImages/

In [8]:
%cd customImages/

/content/customImages


In [0]:
!unzip train.zip

In [0]:
!unzip val.zip

In [0]:
!rm -rf __MACOSX

In [12]:
!ls

main_DataSETS_TRAIN  main_DATASET_VAL  train.zip  val.zip


In [0]:
!mv main_DataSETS_TRAIN/ train

In [0]:
!mv main_DATASET_VAL/ val

In [0]:
!rm -rf train.zip

In [0]:
!rm -rf val.zip

In [17]:
!ls

train  val


In [0]:
!rm -rf __MACOSX

In [19]:
%cd ..

/content


In [20]:
!ls customImages

train  val


In [21]:
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 119.70 MiB | 40.28 MiB/s, done.
Resolving deltas: 100% (555/555), done.


In [0]:
!mv customImages Mask_RCNN/

In [23]:
%cd Mask_RCNN/

/content/Mask_RCNN


In [0]:
!pip install -r requirements.txt

In [0]:
# python setup install
%run setup.py install

In [26]:
!ls

assets	      dist     MANIFEST.in	   README.md	     setup.cfg
build	      images   mask_rcnn.egg-info  requirements.txt  setup.py
customImages  LICENSE  mrcnn		   samples


In [27]:
!wget https://raw.githubusercontent.com/priya-dwivedi/Deep-Learning/master/mask_rcnn_damage_detection/custom.py

--2019-04-23 09:41:27--  https://raw.githubusercontent.com/priya-dwivedi/Deep-Learning/master/mask_rcnn_damage_detection/custom.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13885 (14K) [text/plain]
Saving to: ‘custom.py’

custom.py           100%[===================>]  13.56K  --.-KB/s    in 0.01s   

2019-04-23 09:41:28 (1.34 MB/s) - ‘custom.py’ saved [13885/13885]



In [0]:
!mkdir logs

In [29]:
!ls

assets	      dist     MANIFEST.in	   requirements.txt
build	      images   mask_rcnn.egg-info  samples
customImages  LICENSE  mrcnn		   setup.cfg
custom.py     logs     README.md	   setup.py


In [30]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

MODEL_DIR = os.path.join(ROOT_DIR, "logs")

import custom

%matplotlib inline

Using TensorFlow backend.


In [31]:
config = custom.CustomConfig()
CUSTOM_DIR = os.path.join(ROOT_DIR, "customImages")
print(CUSTOM_DIR)

/content/Mask_RCNN/customImages


In [32]:
# Load dataset
# Get the dataset from the releases page
# https://github.com/matterport/Mask_RCNN/releases
dataset = custom.CustomDataset()
dataset.load_custom(CUSTOM_DIR, "train")

# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

Image Count: 1457
Class Count: 2
  0. BG                                                
  1. damage                                            


In [33]:
class InferenceConfig(custom.CustomConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [0]:
#lists the content of your google drive
# !ls "/content/drive/My Drive/Potholes_tataInnoverse/30"

In [35]:
import shutil
shutil.copy('/content/drive/My Drive/Potholes_tataInnoverse/model_maskRCNN/mask_rcnn_damage_0594.h5', '/content/mask_rcnn_damage_0594.h5')

'/content/mask_rcnn_damage_0594.h5'

In [0]:
!mv /content/mask_rcnn_damage_0594.h5 /content/Mask_RCNN/logs/

In [37]:
!pwd

/content/Mask_RCNN


In [38]:
!ls logs

mask_rcnn_damage_0594.h5


In [39]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights("logs/mask_rcnn_damage_0594.h5", by_name=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
class_names = ['BG', 'damage']

In [41]:
!pip install utils

In [0]:
import os
import sys
import custom
import utils

In [43]:
%cd mrcnn

/content/Mask_RCNN/mrcnn


In [0]:
import model as modellib

In [45]:
%cd ..

/content/Mask_RCNN


In [0]:
import cv2
import numpy as np

In [0]:
## Testing
from PIL import Image, ImageDraw, ImageFont

#### MoveOver for Getting Testing Images Similar to S3 Bucket

In [0]:
#start
## Folder for the S3 Bucket
#lists the content of your google drive
# !ls "/content/drive/My Drive/Potholes_tataInnoverse/S3/"
# !wget https://pradyumgupta.me/Downloads/images.zip
!wget https://github.com/Prady96/Pothole-Detection/blob/avi_testing/test_dataset/images_800.tar.gz?raw=true

In [76]:
!ls

 assets			       LICENSE			     README.md
 build			       logs			     requirements.txt
 customImages		       MANIFEST.in		     S3_Images
 custom.py		       mask_rcnn.egg-info	     samples
 dist			       mrcnn			     setup.cfg
 images			      'OpenSans-Bold.ttf?raw=true'   setup.py
'images_800.tar.gz?raw=true'   __pycache__


In [0]:
!mkdir S3_Images

In [73]:
!ls

 assets         images		     mrcnn			   S3_Images
 build	        LICENSE		    'OpenSans-Bold.ttf?raw=true'   samples
 customImages   logs		     __pycache__		   setup.cfg
 custom.py      MANIFEST.in	     README.md			   setup.py
 dist	        mask_rcnn.egg-info   requirements.txt


In [0]:
!mv images_800.tar.gz?raw=true S3_Images/

In [78]:
%cd S3_Images/

/content/Mask_RCNN/S3_Images


In [79]:
!ls

'images_800.tar.gz?raw=true'


In [0]:
# !unzip images.zip
!mv images_800.tar.gz?raw=true images.tar.gz

In [81]:
!tar xvzf images.tar.gz

images_800/5587.jpg
tar: images_800/5587.jpg: time stamp 2019-04-24 16:27:24 is 110205.442510586 s in the future
images_800/
images_800/4173.jpg
tar: images_800/4173.jpg: time stamp 2019-04-24 16:27:24 is 110205.442016066 s in the future
images_800/IMG_6746.jpg
tar: images_800/IMG_6746.jpg: time stamp 2019-04-24 16:27:24 is 110205.441132193 s in the future
images_800/pgm-bw-112-_98.jpg
tar: images_800/pgm-bw-112-_98.jpg: time stamp 2019-04-24 16:27:24 is 110205.440823635 s in the future
images_800/BulkResizePhotos.html
tar: images_800/BulkResizePhotos.html: time stamp 2019-04-24 16:27:24 is 110205.440765112 s in the future
images_800/IMG_4991.jpg
tar: images_800/IMG_4991.jpg: time stamp 2019-04-24 16:27:24 is 110205.440232562 s in the future
images_800/IMG_4453.jpg
tar: images_800/IMG_4453.jpg: time stamp 2019-04-24 16:27:24 is 110205.439470784 s in the future
images_800/frame3.jpg
tar: images_800/frame3.jpg: time stamp 2019-04-24 16:27:24 is 110205.438790144 s in the future
images_800

In [85]:
!ls images_800 #/content/Mask_RCNN/S3_Images/images_800

171.jpg			   IMG_4991.jpg
351.jpg			   IMG_6746.jpg
4173.jpg		   IMG_9567.jpg
5587.jpg		   Muroran_20170920113720.jpg
Adachi_20170911112605.jpg  Muroran_20170921100101.jpg
BulkResizePhotos.html	   pgm-bw-112-_98.jpg
frame3.jpg		   pgm-bw-115-_145.jpg
IMG_4453.jpg		   test_275.jpg


In [86]:
!pwd

/content/Mask_RCNN/S3_Images


In [0]:
# !rm -rf __MACOSX

In [87]:
!ls

images_800  images.tar.gz


In [0]:
!rm -rf images.tar.gz

In [89]:
!pwd

/content/Mask_RCNN/S3_Images


In [90]:
%cd ..

/content/Mask_RCNN


In [91]:
#end
%cd /content/Mask_RCNN/

/content/Mask_RCNN


In [92]:
!ls

 assets         images		     mrcnn			   S3_Images
 build	        LICENSE		    'OpenSans-Bold.ttf?raw=true'   samples
 customImages   logs		     __pycache__		   setup.cfg
 custom.py      MANIFEST.in	     README.md			   setup.py
 dist	        mask_rcnn.egg-info   requirements.txt


In [93]:
!ls S3_Images

images_800


In [94]:
%cd /content/Mask_RCNN/S3_Images/images_800

/content/Mask_RCNN/S3_Images/images_800


In [0]:
!pip install python-resize-image

In [0]:
from PIL import Image
import os
from resizeimage import resizeimage

count = 0

for f in os.listdir(os.getcwd()):
    f_name, f_ext = os.path.splitext(f)
#     f_random, f_lat_name,f_lat_val,f_long_name,f_long_val = f_name.split('-')
#     f_lat_val = f_lat_val.strip() ##removing the white Space
#     f_long_val = f_long_val.strip()
#     new_name = '{}-{}-{}.jpg'.format(f_lat_val,f_long_val,count)
    try:
        with Image.open(f) as image:
            count +=1
            cover = resizeimage.resize_cover(image, [600,600])
            cover.save('{}{}'.format(f_name,f_ext),image.format)
            #os.remove(f)
            print(count)
    except(OSError) as e:
        print('Bad Image {}{}'.format(f,count))

In [97]:
%cd /content/Mask_RCNN/

/content/Mask_RCNN


In [0]:
!wget https://github.com/Prady96/IITM_PythonTraining/blob/master/ImageWorking_add_textInImage/fonts_Dir/OpenSans-Bold.ttf?raw=true

In [0]:
!mv OpenSans-Bold.ttf?raw=true OpenSans-Bold.ttf

In [100]:
!ls

 assets         logs			        README.md
 build	        MANIFEST.in		        requirements.txt
 customImages   mask_rcnn.egg-info	        S3_Images
 custom.py      mrcnn			        samples
 dist	        OpenSans-Bold.ttf	        setup.cfg
 images        'OpenSans-Bold.ttf?raw=true.1'   setup.py
 LICENSE        __pycache__


In [0]:
# Main file for the file iteration
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
myList = [] ## area list
classList = [] ##class Id List

def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image


def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]
    colors = random_colors(n_instances)

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i, color in enumerate(colors):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image


def save_image(image, image_name, boxes, masks, class_ids, scores, class_names, filter_classs_names=None,
               scores_thresh=0.1, save_dir=None, mode=0):
    """
        image: image array
        image_name: image name
        boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
        masks: [num_instances, height, width]
        class_ids: [num_instances]
        scores: confidence scores for each box
        class_names: list of class names of the dataset
        filter_classs_names: (optional) list of class names we want to draw
        scores_thresh: (optional) threshold of confidence scores
        save_dir: (optional) the path to store image
        mode: (optional) select the result which you want
                mode = 0 , save image with bbox,class_name,score and mask;
                mode = 1 , save image with bbox,class_name and score;
                mode = 2 , save image with class_name,score and mask;
                mode = 3 , save mask with black background;
    """
    mode_list = [0, 1, 2, 3]
    assert mode in mode_list, "mode's value should in mode_list %s" % str(mode_list)

    if save_dir is None:
        save_dir = os.path.join(os.getcwd(), "output")
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

    useful_mask_indices = []

    N = boxes.shape[0]
    if not N:
        print("\n*** No instances in image %s to draw *** \n" % (image_name))
        return
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    for i in range(N):
        # filter
        class_id = class_ids[i]
        score = scores[i] if scores is not None else None
        if score is None or score < scores_thresh:
            continue

        label = class_names[class_id]
        if (filter_classs_names is not None) and (label not in filter_classs_names):
            continue

        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue

        useful_mask_indices.append(i)

    if len(useful_mask_indices) == 0:
        print("\n*** No instances in image %s to draw *** \n" % (image_name))
        return

    colors = random_colors(len(useful_mask_indices))

    if mode != 3:
        masked_image = image.astype(np.uint8).copy()
    else:
        masked_image = np.zeros(image.shape).astype(np.uint8)

    if mode != 1:
        for index, value in enumerate(useful_mask_indices):
            masked_image = apply_mask(masked_image, masks[:, :, value], colors[index])

    masked_image = Image.fromarray(masked_image)

    if mode == 3:
        masked_image.save(os.path.join(save_dir, '%s.jpg' % (image_name)))
        return

    draw = ImageDraw.Draw(masked_image)
    colors = np.array(colors).astype(int) * 255

    myList = []
    countClassIds = 0
    
    for index, value in enumerate(useful_mask_indices):
        class_id = class_ids[value]
        print('class_id value is {}'.format(class_id))
        if class_id == 1:
          countClassIds += 1
        print('counter for the class ID {}'.format(countClassIds))
        
        
        score = scores[value]
        label = class_names[class_id]

        y1, x1, y2, x2 = boxes[value]
        
#         myList = []
        
        ## area of the rectangle
        yVal = y2 - y1
        xVal = x2 - x1
        area = xVal * yVal
        print('area is {}'.format(area))
        myList.append(area)
        
        if mode != 2:
            color = tuple(colors[index])
            draw.rectangle((x1, y1, x2, y2), outline=color)

        # Label
#         font = ImageFont.load('/usr/share/fonts/truetype/ttf-bitstream-vera/Vera.ttf')
        font = ImageFont.truetype('OpenSans-Bold.ttf', 15)
        draw.text((x1, y1), "%s %f" % (label, score), (255, 255, 255), font)

    print(r['class_ids'], r['scores'])
    print(myList)
#     print('value of r is {}'.format(r))
    print('image_name is {}'.format(image_name))

    image_name = os.path.basename(image_name)
    print('image name is {}'.format(image_name))

    f_name, f_ext = os.path.splitext(image_name)
    #f_lat_val,f_long_val,f_count = f_name.split('-')

    #f_lat_val = f_lat_val.strip() ##removing the white Space
    #f_long_val = f_long_val.strip()

#     new_name = '{}-{}-{}.jpg'.format(f_lat_val,f_long_val,count)
#     print([area for area in myList if ])
#       print([i for i in range(countClassIds) ])
      
    print("avi96 {}".format(myList[:countClassIds]))
#     myList.pop(countClassIds - 1)
    
    new_name = '{}-{}.jpg'.format(myList, r['scores'])
#     masked_image.save(os.path.join(save_dir, '%s.jpg' % (image_name)))
    print("New Name file is {}".format(new_name))
    print('save_dir is {}'.format(save_dir))
    masked_image.save(os.path.join(save_dir, '%s' % (new_name)))
    print('file Saved {}'.format(new_name))
#     os.rename(image_name, new_name)



if __name__ == '__main__':
    """
        test everything
    """
    import os
    import sys
    import custom
    import utils
    import model as modellib
    #import visualize

    # We use a K80 GPU with 24GB memory, which can fit 3 images.
    batch_size = 3

    ROOT_DIR = os.getcwd()
    MODEL_DIR = os.path.join(ROOT_DIR, "logs")
    VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
    VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "save")
#     COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_damage_0010.h5")
#     if not os.path.exists(COCO_MODEL_PATH):
#         utils.download_trained_weights(COCO_MODEL_PATH)

    class InferenceConfig(custom.CustomConfig):
        GPU_COUNT = 1
        IMAGES_PER_GPU = batch_size

    config = InferenceConfig()
    config.display()

    model = modellib.MaskRCNN(
        mode="inference", model_dir=MODEL_DIR, config=config
    )
    model.load_weights("logs/mask_rcnn_damage_0594.h5", by_name=True)
    class_names = [
        'BG', 'damage'
    ]

#     capture = cv2.VideoCapture(os.path.join(VIDEO_DIR, 'trailer1.mp4'))
    try:
        if not os.path.exists(VIDEO_SAVE_DIR):
            os.makedirs(VIDEO_SAVE_DIR)
    except OSError:
        print ('Error: Creating directory of data')

    # points to be done before final coding
    """
    path_for_image_dir
    list for the image array
    resolve for naming convention for location basis
    passing image in model
    """

    # path for the data files
    data_path = '/content/Mask_RCNN/S3_Images/images_800/'
    onlyfiles = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))]

    # empty list for the training data
    frames = []
    frame_count = 0
    batch_count = 1

    # enumerate the iteration with number of files
    for j, files in enumerate(onlyfiles):
        image_path = data_path + onlyfiles[j]
#         print("image Path {}".format(image_path))
#         print("Only Files {}".format(onlyfiles[j]))
#         print('j is {}'.format(j))
#         print('files is {}'.format(files))
        try:
            images = cv2.imread(image_path).astype(np.uint8)
#             print("images {}".format(images))
            frames.append(np.asarray(images, dtype=np.uint8))
  #         frames.append(images)
            frame_count += 1
            print('frame_count :{0}'.format(frame_count))
            if len(frames) == batch_size:
                results = model.detect(frames, verbose=0)
                print('Predicted')
                for i, item in enumerate(zip(frames, results)):
#                     print('i is {}'.format(i))
#                     print('item is {}'.format(item))
                    frame = item[0]
                    r = item[1]
                    frame = display_instances(
                      frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
                    )
                    name = '{}'.format(files)
                    name = os.path.join(VIDEO_SAVE_DIR, name)
#                     name = '{0}.jpg'.format(frame_count + i - batch_size)
#                     name = os.path.join(VIDEO_SAVE_DIR, name)
#                   cv2.imwrite(name, frame)
#                     print(name)
                    print('writing to file:{0}'.format(name))
#                     print(name)
                    save_image(images, name, r['rois'], r['masks'], r['class_ids'],
                             r['scores'], class_names, save_dir=VIDEO_SAVE_DIR, mode=0)
                frames = []
                print('clear')
              # clear the frames here

        except(AttributeError) as e:
            print('Bad Image {}'.format(image_path))

    print("Success, check the folder")


"""
    ## Code for the video section
    frames = []
    frame_count = 0
    # these 2 lines can be removed if you dont have a 1080p camera.
    capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)


    while True:
        ret, frame = capture.read()
        # Bail out when the video file ends
        if not ret:
            break

        # Save each frame of the video to a list
        frame_count += 1
        frames.append(frame)
        print('frame_count :{0}'.format(frame_count))
        if len(frames) == batch_size:
            results = model.detect(frames, verbose=0)
            print('Predicted')
            for i, item in enumerate(zip(frames, results)):
                frame = item[0]
                r = item[1]
                frame = display_instances(
                    frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
                )
#                 name = '{0}.jpg'.format(frame_count + i - batch_size)
#                 name = os.path.join(VIDEO_SAVE_DIR, name)
#                 cv2.imwrite(name, frame)
#                 print('writing to file:{0}'.format(name))
                ## add visualise files
#                 visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
#                             class_names, r['scores'])
                save_image(image, name, r['rois'], r['masks'], r['class_ids'],
                                     r['scores'],class_names, save_dir=VIDEO_SAVE_DIR, mode=0)
#                 print(r['class_ids'], r['scores'])

            # Clear the frames array to start the next batch
            frames = []

    capture.release()
"""


In [107]:
!ls

 assets         LICENSE		    'OpenSans-Bold.ttf?raw=true.1'   save.zip
 build	        logs		     __pycache__		     setup.cfg
 customImages   MANIFEST.in	     README.md			     setup.py
 custom.py      mask_rcnn.egg-info   requirements.txt		     videos
 dist	        mrcnn		     S3_Images
 images         OpenSans-Bold.ttf    samples


In [108]:
!ls /content/Mask_RCNN/videos/save/

'[1080, 24640, 1225, 8330]-[0.9986259 0.9895922 0.9852962 0.9636315].jpg'
'[1440, 1210]-[0.9954249 0.9007975].jpg'
'[1640, 1431, 2849, 429, 364, 1083, 1430]-[0.9984597  0.9945833  0.98443395 0.963473   0.95023036 0.9433006'$'\n'' 0.9117861 ].jpg'
'[168, 270, 105, 3276]-[0.9946531  0.9654704  0.96345556 0.91592526].jpg'
'[2700, 540]-[0.99903727 0.9369877 ].jpg'
'[2781, 518]-[0.99927026 0.90588623].jpg'
'[32844, 11781, 1378, 18204]-[0.9984633  0.9945661  0.94000345 0.93494743].jpg'
'[4176]-[0.9218931].jpg'
'[5355, 598]-[0.9951043 0.979298 ].jpg'
'[585]-[0.99658173].jpg'
'[8549, 12648, 10249, 9207]-[0.99725014 0.98441595 0.96100074 0.91744643].jpg'


In [109]:
!zip -r save.zip /content/Mask_RCNN/videos/save/

updating: content/Mask_RCNN/videos/save/ (stored 0%)
  adding: content/Mask_RCNN/videos/save/[2700, 540]-[0.99903727 0.9369877 ].jpg (deflated 1%)
  adding: content/Mask_RCNN/videos/save/[4176]-[0.9218931].jpg (deflated 0%)
  adding: content/Mask_RCNN/videos/save/[32844, 11781, 1378, 18204]-[0.9984633  0.9945661  0.94000345 0.93494743].jpg (deflated 0%)
  adding: content/Mask_RCNN/videos/save/[5355, 598]-[0.9951043 0.979298 ].jpg (deflated 2%)
  adding: content/Mask_RCNN/videos/save/[585]-[0.99658173].jpg (deflated 2%)
  adding: content/Mask_RCNN/videos/save/[1440, 1210]-[0.9954249 0.9007975].jpg (deflated 1%)
  adding: content/Mask_RCNN/videos/save/[1080, 24640, 1225, 8330]-[0.9986259 0.9895922 0.9852962 0.9636315].jpg (deflated 1%)
  adding: content/Mask_RCNN/videos/save/[168, 270, 105, 3276]-[0.9946531  0.9654704  0.96345556 0.91592526].jpg (deflated 0%)
  adding: content/Mask_RCNN/videos/save/[8549, 12648, 10249, 9207]-[0.99725014 0.98441595 0.96100074 0.91744643].jpg (deflated 1%)